In [3]:
import numpy as np

## Fully Connected Layer

![](https://user-images.githubusercontent.com/36406676/52783613-c8490600-3095-11e9-8c80-cf62ec321b03.PNG)

- Fully connected Layer는 마지막 단에서 최종목표를 수행하는 Layer.(Classifying, Regression...)


- 32 X 32 X 3의 이미지가 있으면 1X3072로 펼친다.


- 최종 class개수가 10개 이므로 weight는 3072 X 10으로 만들어 둔 뒤에 input과 weight를 곱하여 10개의 output을 만들어낸다.

## Convolutional Layer

- 합성곱계층(Concolutional Layer)에서는 합성곱 연산을 처리합니다. 합성곱 연산은 이미지 처리에서 말하는 필터연산을 의미합니다.



- 합성곱 연산은 입력 데이터(32 X 32 X 3) 에 필터(5 X 5 X 3)를 적용합니다. 합성곱 연산은 필터의 윈도우를 일정 간격으로 이동해가며 입력 데이터에 적용합니다.



- 입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합을 구합니다.(Wx + b) 그리고 그 결과를 출력의 해당 장소에 저장.

![](https://user-images.githubusercontent.com/36406676/52784061-3641fd00-3097-11e9-8c4d-b278929bf67b.PNG)

- 하나하나 다 곱해주면 28 X 28 X 1 크기의 output이 나옵니다.이 output은 activation map이라고 해서 사진이 가지는 특징을 나타내어 줍니다.


- CNN을 사용할때는 filter를 하나만 사용하지 않습니다. 다양한 필터를 사용하여 필터마다 다른 특징을 나타나게 만드는것.(Blur, Sharp,..). 이 값도 학습의 대상이 되어서 목적(task)를 주면 그에 최적화된 parameter가 저장된다.


- CNN은 local feature, position invariant한 특징을 갖는데 이 conv layer때문이다. filter의 크기 만큼의 정보만 추출하고, 똑같은 좌표가 이동하면서 적용되기 때문에 영상의 여러군데서 나타나는 정보들을 하나로 통합해서 보게된다.

![](https://user-images.githubusercontent.com/36406676/52784645-20353c00-3099-11e9-98bc-0d984a61c5a2.PNG)

- 예를 들어 6개의 필터를 추출한다면 28X28X1의 activation map이 6개가 쌓이므로 28X28X6이 된다.


- 합성곱의 개념은 블록으로 이해하면 쉽다

![](https://user-images.githubusercontent.com/36406676/52902740-4b19be80-3258-11e9-8d26-e215323aa5f5.jpg)

![](https://user-images.githubusercontent.com/36406676/52902741-50770900-3258-11e9-9f14-edb640e98f4d.jpg)

![](https://user-images.githubusercontent.com/36406676/52785131-94241400-309a-11e9-8b1a-96369a51d6a3.PNG)

&nbsp;

- conv layer를 여려겹 겹치면서 반복하게 될 것이다.


- filter의 depth(channel)은 input의 channel과 같아야 한다. 예를 들어 아래 이미지에서 28X28X6의 이미지에 필터를 씌우려면 가로 세로 길이는 임의로 정하면 될 것이고 깊은 6이여야 합니다. input의 깊이가 6이기 때문입니다.


- 28X28X6의 이미지에 5X5X6의 필터가 10개가 들어간다면 activation map의 크기는 24X24X10이 될것입니다. 필터1번에 activation map 1겹이 쌓이게 됩니다.

&nbsp;

![](https://user-images.githubusercontent.com/36406676/52785333-0dbc0200-309b-11e9-88bc-9c2416d4c8d0.PNG)

&nbsp;

input 데이터에 필터를 적용하려면 필터가 적용되는 부분을 일자로 펴서 wx+b하기 좋게 만들어줘야한다. 이러한 기능을 하는 함수 img2col을 만들어주었다.


&nbsp;

In [5]:
# convolution을 정의하기 전에 im2col이라는 함수 생성
# im2col은 입력 데이터를 필터링 하기 좋게 전개하는 함수.
# 3차원 입력 데이터에 im2col을 적용하면 2차원 행렬로 바뀝니다.

#img2col
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col



![](https://user-images.githubusercontent.com/36406676/52911283-fbcd9f80-32e4-11e9-8c33-cb0a991fc5e5.jpg)

![](https://user-images.githubusercontent.com/36406676/52911295-1c95f500-32e5-11e9-8ea6-4fe46a7510fe.jpg)

In [6]:
# 실제로 사용
x1 = np.random.rand(1,3,7,7) #(데이터 수, 채널 수 , 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad = 0)
print(col1.shape) # stride 1이므로 3 X 3 = 9, 5X5X3 = 75 . 9번의 행렬곱이 이뤄져야함.
x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2, 5, 5, stride = 1, pad = 0)
print(col2.shape) # stride 1, 데이터 수 10개   3X3X10 = 90 , 5X5X3 = 75 

(9, 75)
(90, 75)


Conv Layer도 구현해보았다. 아래 사진 내용을 직접 대입해보았다.

In [7]:
class Convolution:
    def __init__(self,w,b,stride=1, pad= 0):
        self.w = w
        self.b = b
        self.stride = stride
        self.pad = pad
    
        
        #중간 데이터 (backward시 사용)
        self.x = None
        self.col = None
        self.col_w = None
        
        # 가중치와 편향 매개변수의 기울기
        self.dw = None
        self.db = None
    
    def forward(self,x):
        FN, C, FH, FW = self.w.shape # FN:  필터개수, C: 채널수, FH: 필터높이. FW:필터너비
        N, C, H, W = x.shape # N개의 데이터
        out_h = int(1+(H+ 2*self.pad - FH) / self.stride)
        out_w = int(1+(W+ 2*self.pad - FW) / self.stride) # im2col로 펴진 데이터를 reshape위해
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.w.reshape(FN, -1).T # 필터전개
        out = np.dot(col,col_W) + self.b # (28X28, 6) = (784,6)
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2) # (N, H, W, C) -> (N, C, H, W)
        
        return out
    
    def backward(self, dout):
        FN, C, FH, FW = self.w.shape
        dout = dout.transpose(0,2,3,1).reshape(-1,FN)
        
        self.db = np.sum(dout, axis=0)
        self.dw = np.dot(self.col.T, dout)
        self.dw = self.dw.transpose(1,0).reshape(FN, C, FH, FW)
        
        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.s.shape, FH, FW, self.stride, self.pad)
        
        return dx

![](https://user-images.githubusercontent.com/36406676/52162205-9f2e8a00-2713-11e9-94e9-8984011f972a.PNG)

In [8]:
# random.randn으로 w,b,x 임의로 지정, batch_size = 1
x = np.random.rand(1, 3, 32, 32)  # 32X32X3 input에 5X5X3의 필터를 대입
w = np.random.rand(6, 3, 5, 5)    # filter 수 6개
b = np.random.rand(784,6)         # output size가 28X28 = 784 

conv = Convolution(w,b)
conv.forward(x).shape             # (1,6,28,28)

(1, 6, 28, 28)

## Stride

- 아래와 같이 7X7 input에 3X3 filter를 적용하고 싶다면 위아래로 이동해야되는데 이동하는 stepsize가 stride입니다. stride가 1이면 오른쪽으로 한칸 씩 다섯번 이동할 것이고 아래쪽으로 한칸씩 다섯번 이동할 것입니다. 


- 이를 수식으로 정리하면 output size : **(N-F) / stride + 1 **


- stride = 2일 경우 output size는? (7-3)/2 +1 = 3이므로 3X3

![](https://user-images.githubusercontent.com/36406676/52786445-30034f00-309e-11e9-844c-8c3320261309.PNG)

## Zeropad

- 하지만 stride가 3인 경우 나누어 떨어지지 않는다. (N-F)/stride +1 = 3/4 +1이 된다. 이경우 zero padding을 이용한다. padding이란 이미지의 가장자리 부분에 어떠한 숫자들을 채워 넣는 것을 의미한다. 가장자리에 0을 넣은 것을 zero padding이라고 한다. 


- 공식은 **(N-F+2P)/stride +1** 이 된다. P는 zero padding의 수


- 보통 패딩은 filter_size가 FXF일때  (F-1)/2 만큼 준다.

![](https://user-images.githubusercontent.com/36406676/52787019-f3385780-309f-11e9-9c65-3f55fdcfbc0c.PNG)

아래 그림에서 F=3(pad=1), F=5(pad=2), F=7(pad=3)의 output_size도 직접 구해보았다

![](https://user-images.githubusercontent.com/36406676/52167071-c0fe3000-2758-11e9-829f-6b0c74d50b53.PNG)

In [9]:
## 이번에는 padding을 입혀보자

x = np.random.rand(1,3,7,7)
w = np.random.rand(6,3,3,3) # 6 filter 3X3 filter
b = np.random.rand(49,6)      
conv = Convolution(w,b,pad=1)       
conv.forward(x).shape # 7x7 output!

(1, 6, 7, 7)

In [10]:
## 다른것도 해보자.
x2 = np.random.rand(1,3,7,7)
w2 = np.random.rand(6,3,5,5) # 6filter 5X5 filter
b2 = np.random.rand(49,6)      
conv2 = Convolution(w2,b2,pad=2)  
conv2.forward(x2).shape # 7x7 output!

(1, 6, 7, 7)

In [11]:
## 다른것도 해보자.
x3 = np.random.rand(1,3,7,7)
w3 = np.random.rand(6,3,7,7) # 6filter 7X7 filter
b3 = np.random.rand(49,6)      
conv3 = Convolution(w3,b3,pad=3)  
conv3.forward(x3).shape # 7x7 output!

(1, 6, 7, 7)

&nbsp;

패딩을 하는 또 다른 이유는 input의 크기를 보전하는대도 있습니다. 

convolution의 과정에서 사진의 크기를 그대로 유지시켜 주는 역할을 padding이 해줍니다.

다음 퀴즈를 풀어봅시다. 직접 구해보았습니다.

&nbsp;

![](https://user-images.githubusercontent.com/36406676/52787577-a5bcea00-30a1-11e9-85d5-55628767b51f.PNG)

In [12]:
## input volume: 32X32X3 filter 5*5
x4 = np.random.rand(1,3,32,32)
w4 = np.random.rand(10,3,5,5)               # 10개의 5X5 filter
b4 = np.random.rand(1024,10)                # 32X32 = 1024(filter_size 32x32) 
conv4 = Convolution(w4,b4,pad=2, stride=1)  
conv4.forward(x4).shape                     # (1,10,32,32)

(1, 10, 32, 32)

&nbsp;

32 X 32 X 3의 input 이미지 그리고 filter size 는 5 X 5 X 10.filter가 10개.

output size는 (32-5+2*2)/1 +1 = 32 즉 **32 X 32 X 10**

Conv Layer를 거쳤지만 사진의 크기가 그대로 유지되었습니다. 이것이 패딩을 하는 이유

그렇다면 위의 layer에서 parameter의 개수는 몇개나 될까요??? 

**5 X 5 X 3 + 1(bias) X 10 = 760**입니다.

&nbsp;

![](https://user-images.githubusercontent.com/36406676/52788144-655e6b80-30a3-11e9-94d3-fbedaf68e93d.PNG)

&nbsp;

지금까지 한 내용들을 정리해 보겠습니다. 

W1 X H1 X D1의 크기인 input이 있고(W=가로, H =세로, D = 깊이) 

k는 필터의 개수, 필터size인 F(FXF), S는 stride의 크기, P는 padding의 수이다. 이 필터가 적용되면 output size는 


__W2 = (W1 - F + 2XP)/S +1__

__H2 = (H2 - F + 2XP)/S +1__

__D2 = K__ 인 output값 W2 X H2 X D2가 만들어집니다.

그리고 parameter의 개수는 __(F\*F\*D1)\*K +K__ 가 될 것입니다.

&nbsp;
&nbsp;


- Convolutional layer parameter에 대한 설정 tip

filter 개수는 2의 n승(ex. 32, 64, 128, 512...)

filter 가로(세로)길이 = 3, stride =1, zero pad = 1

filter 가로(세로)길이 = 5, stride =1, zero pad = 2

filter 가로(세로)길이 = 5, stride =2, zero pad = ? (맞는거 아무거나)

filter 가로(세로)길이 = 3, stride =1, zero pad = 0

&nbsp;


![](https://user-images.githubusercontent.com/36406676/52788806-67292e80-30a5-11e9-91b0-073dcbbe0c61.PNG)

위 사진은 filter의 크기가 1X1 인 경우로 W, H는 그대로 유지되고 Depth만 축소됩니다. 

padding이 없어도 크기가 그대로 유지되며 activation map도 아무런 탈도 없이 나옵니다.

이러한 F=1을 사용하는 Convnet을 다음장(9장) 에서 배운다.

## Pooling Layer
convolutional layer와 relu function을 거친 뒤, 처리와 관리를 쉽게 하도록 만들어주는 것이 pooling layer. Downsampling 이라고도 볼 수 있다. 

아래 사진을 보면 224의 filter_size가 112로 줄어든 것 확인

conv layer가 쌓이면 activation의 깊이가 깊어져서 224\*224\*64같은 크기의 이미지가 만들어지고 계산량이 많아진다. 따라서 pooling을 해서 activation map의 크기를 반으로 팍 줄여준다면 계산도 두 배 빨라질 것이다.

![](https://user-images.githubusercontent.com/36406676/52789163-7361bb80-30a6-11e9-850e-08559f14eeaa.PNG)

풀링의 가장 대표적인 방법으로는 MAXPOOLING이 있습니다. 

우선 pooling layer의 filter를 준비합니다. 이때 주의할점은 이 filter에는 parameter들이 없습니다. 아래 예에서 4\*4 크기의 사진에 2\*2크기의 filter로, stride는 2로 pooling을 합니다.

그리고 fitler를 convolution 하듯이 이미지에다가 max연산을 해줍니다. 예를 들면 빨간 부분안에서 가장 큰 6값만 남겨지고 나머지는 지웁니다. 그 다음 stride가 2이므로 오른쪽으로 두 칸 움직여서 다시 초록색부분에서 가장 큰 값인 8만 남겨두고 다 지웁니다.

여기서 주의해야 할 점은 일반적으로 stride는 filter끼리 겹치는 것을 지양해야 합니다. 

![](https://user-images.githubusercontent.com/36406676/52173225-8f6e7e80-27c3-11e9-878a-f0a7a35c0b23.PNG)

In [181]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride = 1, pad = 0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape # (batchsize, colour, height, weight)
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        #전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        #최댓값(2)
        out = np.max(col, axis=1)
        
        out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)
        
        return out
    
    def backward(self, x):
        dout = dout.transpose(0,2,3,1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.s.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx
        

In [183]:
# 위 그림대로  따라해봄
x = np.array([[[[1,1,2,4],
                [5,6,7,8],
                [3,2,1,0],
                [1,2,3,4]]]])

Pool = Pooling(2,2,stride=2) 
Pool.forward(x)

In [182]:
x = np.random.randn(1,3,4,4)
Pool = Pooling(2,2,stride=2) 
Pool.forward(x) # 2X2가 depth 인 3만큼 있음

array([[[[ 1.41884088,  1.15983335],
         [ 1.99876974, -0.22019957]],

        [[ 1.29151128,  1.14541837],
         [ 1.35793363,  0.54427693]],

        [[ 0.8756976 ,  0.62052864],
         [ 2.57428995,  0.7788195 ]]]])

In [ ]:
simple Convnet을 구현해봄

## Simple Convnet

In [293]:
import sys, os
sys.path.append(os.pardir)
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient

class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param={'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},              # Conv layer는 30X5X5의 filter, pad는 0, stride=1의 과정을 거친다.
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        self.params = {
            'W1': weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size),
            'b1': np.zeros(filter_num),
            'W2': weight_init_std * np.random.randn(pool_output_size, hidden_size),
            'b2': np.zeros(hidden_size),
            'W3': weight_init_std * np.random.randn(hidden_size, output_size),
            'b3': np.zeros(output_size),
        }
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],                       # Conv - Pool - FC - FC의 단계를 거침
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)                                   # Pooling의 filter_size는 2X2 , stride는 2
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
            
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)
            
        return acc / x.shape[0]
    
    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)
        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])
            
        return grads

    def gradient(self, x, t):
        self.loss(x, t)

        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {
            'W1': self.layers['Conv1'].dW,
            'b1': self.layers['Conv1'].db,
            'W2': self.layers['Affine1'].dW,
            'b2': self.layers['Affine1'].db,
            'W3': self.layers['Affine2'].dW,
            'b3': self.layers['Affine2'].db,
        }

        return grads

    def save_params(self, file_name='params.pkl'):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name='params.pkl'):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]


In [294]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# if you want to reduce training time, apply following lines.
x_train = x_train[:5000]
t_train = t_train[:5000]
x_test = x_test[:1000]
t_test = t_test[:1000]

In [296]:
max_epochs = 5

network = SimpleConvNet(input_dim=(1, 28, 28),
                        conv_param={'filter_num': 30,
                                    'filter_size': 5,
                                    'pad': 0,
                                    'stride': 1,
                                   }
                       )

trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000,
                 )

trainer.train()

train loss:2.2995731609903176
=== epoch:1, train acc:0.223, test acc:0.197 ===
train loss:2.2969068238454633
train loss:2.292434397173415
train loss:2.2854925535874178
train loss:2.27490929461273
train loss:2.266689276941388
train loss:2.2669428702039927
train loss:2.248037432034485
train loss:2.226887780492325
train loss:2.2080510662512736
train loss:2.1736515465778075
train loss:2.126932162280348
train loss:2.103144222055123
train loss:2.0587359987048526
train loss:2.001569781615461
train loss:1.9363298946922223
train loss:1.7673470029529674
train loss:1.8032646809871629
train loss:1.77526206612391
train loss:1.6616556572498298
train loss:1.6114805424625935
train loss:1.4169475618058829
train loss:1.4212768711279977
train loss:1.2815824564074838
train loss:1.2283695657740141
train loss:1.1590456353665288
train loss:1.029974848252532
train loss:1.006837707656893
train loss:0.9172189970478394
train loss:0.9914255228465175
train loss:0.978259520707926
train loss:0.7780486914101309
train